In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import MarianMTModel, MarianTokenizer


In [2]:
file_path = 'LK_modified.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)
dfs = {sheet_name: pd.DataFrame(sheet_data) for sheet_name, sheet_data in all_sheets.items()}
popular_phrases = dfs[list(dfs.keys())[0]]
df = dfs[list(dfs.keys())[1]]
glossary = dfs[list(dfs.keys())[2]]

In [3]:
glossary

,Сокращение,Расшифровка
0,лк,личный кабинет
1,БиР,Беременность и роды
2,зп,заработная плата
3,НДФЛ,Налог на доходы физических лиц
4,СТД,срочный трудовой договор
5,ТК,трудовой договор
6,АО,авансовый отчет
7,SLA,сроки
8,ЭЦП,электронная цифровая подпись
9,КР,кадровый резерв


In [4]:
src_lang = "Helsinki-NLP/opus-mt-ru-en"
tgt_lang = "Helsinki-NLP/opus-mt-en-ru"

tokenizer_src = MarianTokenizer.from_pretrained(src_lang)
model_src = MarianMTModel.from_pretrained(src_lang)

tokenizer_tgt = MarianTokenizer.from_pretrained(tgt_lang)
model_tgt = MarianMTModel.from_pretrained(tgt_lang)

C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
answer = df['content'][0]
category = df['category'][0]
sentence = df['question'][0]
sentence

'Как заказать выпуск МЧД в ТС5?'

In [6]:
def backtranslate_with_temperature(sentence, num_variations=5, temperature=1):
    inputs = tokenizer_src(sentence, return_tensors="pt")
    translated = model_src.generate(**inputs, temperature=temperature, num_return_sequences=num_variations)

    generated_sentences = []
    for translation in translated:
        translated_sentence = tokenizer_src.decode(translation, skip_special_tokens=True)
        inputs_back = tokenizer_tgt(translated_sentence, return_tensors="pt")
        back_translated = model_tgt.generate(**inputs_back, temperature=temperature)
        final_sentence = tokenizer_tgt.decode(back_translated[0], skip_special_tokens=True)
        generated_sentences.append(final_sentence)

    return generated_sentences

generated_sentences = backtranslate_with_temperature(sentence, num_variations=5, temperature=1)

In [7]:
for i, sent in enumerate(generated_sentences, 1):
    print(f"Вариант {i}: {category} {sent}")

Вариант 1: ЭЦП Как вы закажете медэкспертизу по TC5?
Вариант 2: ЭЦП Как вы заказываете медэкспертов в TC5?
Вариант 3: ЭЦП Как вы заказываете медэкспертов в TC5?
Вариант 4: ЭЦП Как вы закажете медосмотр в TC5?
Вариант 5: ЭЦП Как вы заказываете медэкспертов на TS5?


In [11]:
answer = df['content'][6]
category = df['category'][6]
sentence = df['question'][6]
sentence

'больничный по уходу за членом семьи'

In [12]:
generated_sentences = backtranslate_with_temperature(sentence, num_variations=5, temperature=1.2)
for i, sent in enumerate(generated_sentences, 1):
    print(f"Вариант {i}: {category} {sent}")

C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Вариант 1: уход за больным Больница для члена семьи
Вариант 2: уход за больным Больница для ухода за членом семьи
Вариант 3: уход за больным Больничное обслуживание члена семьи
Вариант 4: уход за больным Больница для членов семьи
Вариант 5: уход за больным Больница для ухода за членом семьи


In [79]:
df['content_label'] = df['content'].factorize()[0]

In [80]:
df

,id,question,content,category,content_label
0,1577,как заказать выпуск мчд в тс5?,1. Заходим на портал поддержки https://company...,ЭЦП,0
1,1225,сотруднику в личный кабинет не приходит ссылич...,Анкета СБ приходит при первичном трудоустройстве,СБ,1
2,1560,как получить выплату по уходу за больным родст...,Больничный с кодом 09 (уход за больным членом ...,уход за больным,2
3,1561,как получить выплату по уходу за больным родст...,Больничный с кодом 09 (уход за больным членом ...,уход за больным,2
4,1562,почему нет выплаты от работодателя по больничн...,Больничный с кодом 09 (уход за больным членом ...,уход за больным,2
...,...,...,...,...,...
1683,1686,вернуться на старый портал,Чат-бот находится в стадии пилотирования и обу...,поддержка,215
1684,1687,нужен оператор,Чат-бот находится в стадии пилотирования и обу...,поддержка,215
1685,1580,нет задачи на подписание графика,Чтобы дочерняя задача появилась на подписи у с...,ЭЦП,216
1686,1403,где взять шаблон заявления на увольнение?,Шаблон размещен в Базе знаний Личного кабинета...,увольнение,217


In [83]:
rare_categories = df['content'].value_counts()[df['content'].value_counts() <= 4].index
non_rare = df['content'].value_counts()[df['content'].value_counts() > 4].index
rare_data = df[df['content'].isin(rare_categories)]
df = df[df['content'].isin(non_rare)]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    df['content_label'],
                                                    test_size=0.20,
                                                    stratify=df['content_label'],
                                                    random_state=42)

In [86]:
X_train


,id,question,content,category,content_label
1174,545,сотрудник удалил нечаянно логин и пароль от ли...,"При проблемах со входом в личный кабинет, преж...",ЛК,169
1562,1123,не приходит заявка из скиллаз в личный кабинет2,"Создайте, пожалуйста, обращение в ИТ поддержку...",поддержка,191
79,250,отсутствует график работы,"Вкладка ""график работы"" недоступна сотрудникам...",ЛК,26
1042,413,как сбросить пароль,"При проблемах со входом в личный кабинет, преж...",ЛК,169
1329,64,изменить данные пособия,"Создайте заявку по теме «Больничный», подтема ...",больничный,189
...,...,...,...,...,...
328,1247,нет моей команды в личный кабинет. с уважением дм,"Если в ""команде"" нет подчиненных сотрудников п...",табель,80
1106,477,директор не может зайти в личный кбинет сотруд...,"При проблемах со входом в личный кабинет, преж...",ЛК,169
11,1226,где забрать мою справку,"В заявке на заказ справки Вы указываете, где х...",справка,3
275,659,"нет доступа к программе ""моя карьера""","Доступ к программе ""карьера"" появляется спустя...",моя карьера,77


In [94]:
AUG_NUM = 5

def balance_dataset(qa_df):
    # Шаг 1: Найти самый частовстречаемый ответ
    max_count = qa_df['content_label'].value_counts().max()

    # Шаг 2: Сбалансировать выборку ответов
    augmented_data = []
    for content_id, group in tqdm(qa_df.groupby('content_label')):
        count = len(group)
        augmented_data.extend(group.to_dict('records'))  # Добавляем все исходные строки

        # Если ответ встречается реже, чем самый частовстречаемый, создаем аугментированные копии вопросов
        for _ in range(min(AUG_NUM, max_count - count)):
            row = group.sample(1).iloc[0].to_dict()  # Случайный вопрос из группы
            question = row['question']

            # Применяем несколько аугментаций последовательно
            augmented_question = backtranslate_with_temperature(question, num_variations=5, temperature=1)
            #augmented_question = generate_paraphrases(question, num_return_sequences=5)
            #augmented_question = add_or_remove_punctuation(question)
            #augmented_question = introduce_typo(augmented_question)
            #augmented_question = shuffle_words(augmented_question)

            # Сохраняем аугментированный вопрос с исходным ответом
            new_row = row.copy()
            new_row['question'] = augmented_question
            new_row['category'] = row['category']
            augmented_data.append(new_row)

    # Шаг 3: Создать новый сбалансированный датафрейм
    balanced_qa_df = pd.DataFrame(augmented_data)
    return balanced_qa_df

In [95]:
balanced_qa_df=balance_dataset(X_train)

100%|██████████| 63/63 [21:40<00:00, 20.64s/it]


долго(

In [96]:
X_train['question'] = X_train['category'] + " " + X_train['question']
X_test['question'] = X_test['category'] + " " + X_test['question']
X_train['question'] = X_train['question'].str.lower()
X_test['question'] = X_test['question'].str.lower()

In [97]:
X_train = X_train['question']
X_test = X_test['question']

In [111]:
X_train

1174    лк сотрудник удалил нечаянно логин и пароль от...
1562    поддержка не приходит заявка из скиллаз в личн...
79                           лк отсутствует график работы
1042                               лк как сбросить пароль
1329                   больничный изменить данные пособия
                              ...                        
328     табель нет моей команды в личный кабинет. с ув...
1106    лк директор не может зайти в личный кбинет сот...
11                        справка где забрать мою справку
275     моя карьера нет доступа к программе "моя карьера"
54      удаленная работа не могу изменить и создать но...
Name: question, Length: 1135, dtype: object

In [100]:
### tfidf
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [101]:
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
y_pred = rf_classifier.predict(X_test_tfidf)

In [102]:
# Подсчет метрик
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_micro = f1_score(y_test, y_pred, average='micro')

# Вывод метрик
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: (macro:{precision_macro:.4f}, micro:{precision_micro:.4f})')
print(f'Recall: (macro:{recall_macro:.4f}, micro:{recall_micro:.4f})')
print(f'F1 Score: (macro:{f1_macro:.4f}, micro:{f1_micro:.4f}) \n')

Accuracy: 0.8803
Precision: (macro:0.7336, micro:0.8803)
Recall: (macro:0.7227, micro:0.8803)
F1 Score: (macro:0.7140, micro:0.8803) 



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
